# Datos Sinteticos Contactabilidad Vanti

In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

# Configuraciones iniciales
def configurar_entorno():
    fake = Faker('es_CO')
    Faker.seed(0)
    np.random.seed(0)
    random.seed(0)
    return fake

fake = configurar_entorno()

# Parámetros de generación de datos
NUM_CLIENTS = 500
NUM_CONTACTS = 800
NUM_RESPONSES = 800
DATE_START = datetime(2019, 2, 17)
DATE_END = datetime(2024, 11, 14)

# Distribuciones ajustables
DISTRIBUCIONES = {
    'nivel_educativo': {
        'Primaria': 30,
        'Secundaria': 40,
        'Universidad': 25,
        'Posgrado': 4,
        'Doctorado': 1
    },
    'estado_civil': {
        'Soltero': 50,
        'Casado': 30,
        'Divorciado': 10,
        'Viudo': 5,
        'Unión libre': 5
    },
    'ocupaciones': {
        'Administrativo': 20,
        'Técnico': 15,
        'Ingeniero': 10,
        'Profesor': 10,
        'Médico': 5,
        'Abogado': 5,
        'Contador': 10,
        'Arquitecto': 5,
        'Enfermero': 10,
        'Desarrollador': 10
    },
    'tipos_contacto': {
        'Atención de Emergencias': 20,
        'Revisión Periódica Obligatoria (RPO)': 15,
        'Atención Comercial': 10,
        'Reclamos y PQR': 10,
        'Actualización de Datos': 10,
        'Campañas de Seguridad': 5,
        'Autogestión de Servicios': 10,
        'Información sobre Facturación': 10,
        'Avisos de Mantenimiento': 5,
        'Consultas Técnicas': 5
    },
    'satisfaccion': {
        1: 5,
        2: 10,
        3: 20,
        4: 40,
        5: 25
    },
    'respuesta': {
        'Sí': 70,
        'No': 5,
        'No contesta': 20,
        'Dejó mensaje': 5
    }
}

# Validar distribuciones
def verificar_porcentajes(diccionario, nombre_campo):
    total_porcentaje = sum(diccionario.values())
    if total_porcentaje != 100:
        raise ValueError(f"Los porcentajes de {nombre_campo} deben sumar 100%. Actualmente suman {total_porcentaje}%.")

for nombre, distribucion in DISTRIBUCIONES.items():
    verificar_porcentajes(distribucion, nombre)

# Generar fechas aleatorias
def random_date(start, end):
    if isinstance(start, (int, float)):
        start = datetime.fromtimestamp(start)
    if isinstance(end, (int, float)):
        end = datetime.fromtimestamp(end)
    delta = end - start
    if delta.days > 0:
        random_days = random.randint(0, delta.days)
        return start + timedelta(days=random_days)
    else:
        return start


# Generación de datos para clientes
def generar_clientes(num_clients, fake):
    clientes = []
    sexos = ['Masculino', 'Femenino', 'No especificado']
    ciudades = [
        ('Bogotá', 'Bogotá', 20),
        ('Soacha', 'Cundinamarca', 5),
        ('Sibaté', 'Cundinamarca', 2),
        ('La Calera', 'Cundinamarca', 2),
        ('Fusagasugá', 'Cundinamarca', 3),
        ('Girardot', 'Cundinamarca', 3),
        ('Zipaquirá', 'Cundinamarca', 2),
        ('Facatativá', 'Cundinamarca', 2),
        ('Chía', 'Cundinamarca', 2),
        ('Cajicá', 'Cundinamarca', 2),
        ('Mosquera', 'Cundinamarca', 2),
        ('Madrid', 'Cundinamarca', 2),
        ('Funza', 'Cundinamarca', 2),
        ('Villeta', 'Cundinamarca', 1),
        ('Tunja', 'Boyacá', 3),
        ('Duitama', 'Boyacá', 3),
        ('Sogamoso', 'Boyacá', 3),
        ('Chiquinquirá', 'Boyacá', 2),
        ('Paipa', 'Boyacá', 2),
        ('Bucaramanga', 'Santander', 5),
        ('Floridablanca', 'Santander', 2),
        ('Girón', 'Santander', 2),
        ('Piedecuesta', 'Santander', 2),
        ('Barrancabermeja', 'Santander', 2),
        ('San Gil', 'Santander', 1),
        ('Aguachica', 'Cesar', 1),
        ('Gamarra', 'Cesar', 1),
        ('La Gloria', 'Cesar', 1),
        ('San Alberto', 'Cesar', 1),
        ('San Martín', 'Cesar', 1)
    ]
    for i in range(1, num_clients + 1):
        ciudad, departamento, peso = random.choices(ciudades, weights=[ciudad[2] for ciudad in ciudades], k=1)[0]
        cliente = {
            'ID Cliente': i,
            'Fecha de creación': random_date(DATE_START, DATE_END),
            'Identificación': str(random.randint(10000000, 99999999)),
            'Nombre': fake.name(),
            'Fecha de nacimiento': fake.date_of_birth(minimum_age=18, maximum_age=90),
            'Sexo': random.choice(sexos),
            'Nivel educativo': random.choices(list(DISTRIBUCIONES['nivel_educativo'].keys()), weights=list(DISTRIBUCIONES['nivel_educativo'].values()), k=1)[0],
            'Ocupación': random.choices(list(DISTRIBUCIONES['ocupaciones'].keys()), weights=list(DISTRIBUCIONES['ocupaciones'].values()), k=1)[0],
            'Estado civil': random.choices(list(DISTRIBUCIONES['estado_civil'].keys()), weights=list(DISTRIBUCIONES['estado_civil'].values()), k=1)[0],
            'Ciudad': ciudad,
            'Departamento': departamento
        }
        clientes.append(cliente)
    return pd.DataFrame(clientes)

df_clientes = generar_clientes(NUM_CLIENTS, fake)

# Introducir errores aleatorios en la identificación
def introducir_errores_identificacion(df, porcentaje_errores):
    num_errors = int(porcentaje_errores * len(df))
    for _ in range(num_errors):
        idx = random.randint(0, len(df) - 1)
        df.at[idx, 'Identificación'] = ''  # Introducir error en la identificación
    return df

df_clientes = introducir_errores_identificacion(df_clientes, 0.05)

# Funciones auxiliares para generación de datos

def generate_colombian_mobile():
    prefix = random.randint(300, 324)
    number = random.randint(1000000, 9999999)
    return f"+57 {prefix} {number}"

def generate_address(ciudad, departamento):
    barrios = {
        'Bogotá': [
            'Chapinero', 'Teusaquillo', 'Usaquén', 'Suba', 'Engativá', 'Fontibón',
            'Kennedy', 'Bosa', 'Puente Aranda', 'San Cristóbal', 'Santa Fe', 'Usme',
            'Ciudad Bolívar', 'Rafael Uribe Uribe', 'Tunjuelito', 'Antonio Nariño',
            'La Candelaria', 'Barrios Unidos', 'Los Mártires'
        ],
        # (Otros barrios para cada ciudad omitidos por brevedad)
    }
    tipos_via = ['Calle', 'Carrera', 'Avenida', 'Transversal', 'Diagonal']
    tipo_via = random.choice(tipos_via)
    numero_via = random.randint(1, 150)
    letra_via = random.choice(['', 'A', 'B', 'C'])
    numero_secundario = random.randint(1, 100)
    placa = random.randint(1, 50)
    barrio = random.choice(barrios.get(ciudad, ['Centro']))
    return f"{tipo_via} {numero_via}{letra_via} #{numero_secundario}-{placa}, {barrio}, {ciudad}, {departamento}, Colombia"

# Generación de canales de contacto
def generar_canales_contacto(df_clientes, fake):
    canales = []
    for _, cliente in df_clientes.iterrows():
        for tipo in ['Email', 'Teléfono', 'Dirección', 'WhatsApp']:
            canal = {
                'ID Canal': len(canales) + 1,
                'ID Cliente': cliente['ID Cliente'],
                'Fecha de actualización': random_date(cliente['Fecha de creación'], DATE_END),
                'Canal de actualización': random.choices(['Oficina Virtual', 'App móvil', 'Teléfono', 'Presencial', 'WhatsApp'], weights=[30, 25, 20, 10, 15], k=1)[0],
                'Estado': random.choices(['Activo', 'Inactivo'], weights=[95, 5], k=1)[0],
                'Tipo': tipo,
                'Valor': generar_valor_canal(tipo, fake, cliente['Ciudad'], cliente['Departamento']),
                'Validación': random.choices([True, False], weights=[85, 15], k=1)[0],
                'Consentimiento': random.choices([True, False], weights=[80, 20], k=1)[0]
            }
            canales.append(canal)
    return pd.DataFrame(canales)

def generar_valor_canal(tipo, fake, ciudad, departamento):
    if tipo == 'Email':
        return fake.email()
    elif tipo == 'Teléfono' or tipo == 'WhatsApp':
        return generate_colombian_mobile()
    else:
        return generate_address(ciudad, departamento)

df_canales = generar_canales_contacto(df_clientes, fake)

# Generación de contactabilidad
def generar_contactabilidad(num_contacts, df_clientes, df_canales):
    contactabilidad = []
    client_channels = df_canales.groupby('ID Cliente')['ID Canal'].apply(list).to_dict()
    for i in range(1, num_contacts + 1):
        id_cliente = random.randint(1, NUM_CLIENTS)
        tipo_contacto = random.choices(list(DISTRIBUCIONES['tipos_contacto'].keys()), weights=list(DISTRIBUCIONES['tipos_contacto'].values()), k=1)[0]
        canal_tipo = random.choice(['Email', 'Teléfono', 'Dirección', 'WhatsApp'])
        id_canal = random.choice(client_channels[id_cliente])
        fecha_actualizacion_canal = pd.to_datetime(df_canales.loc[df_canales['ID Canal'] == id_canal, 'Fecha de actualización'].values[0])
        iniciador_contacto = random.choices(['Vanti', 'Cliente'], weights=[70, 30], k=1)[0]
        respuesta = random.choices(list(DISTRIBUCIONES['respuesta'].keys()), weights=list(DISTRIBUCIONES['respuesta'].values()), k=1)[0]

        # Generar múltiples intentos de contacto si el cliente no responde
        num_intentos = random.randint(1, 3) if respuesta == 'No contesta' else 1
        for intento in range(num_intentos):
            contacto_fecha = random_date(fecha_actualizacion_canal, DATE_END)
            contacto = {
                'ID Contacto': len(contactabilidad) + 1,
                'ID Canal': id_canal,
                'Fecha': contacto_fecha,
                'Canal': canal_tipo,
                'Tipo de contacto': tipo_contacto,
                'Iniciador del Contacto': iniciador_contacto,
                'Respuesta': 'Sí' if intento == num_intentos - 1 and random.random() < 0.5 else ('Dejó mensaje' if intento == num_intentos - 1 and respuesta == 'No contesta' and random.random() < 0.5 else 'No contesta'),
                'Satisfacción': random.choices(list(DISTRIBUCIONES['satisfaccion'].keys()), weights=list(DISTRIBUCIONES['satisfaccion'].values()), k=1)[0]
            }
            contactabilidad.append(contacto)

        # Contacto de respuesta del cliente a Vanti (bidireccionalidad)
        if random.random() < 0.5 and iniciador_contacto == 'Vanti' and respuesta == 'Sí':  # Solo si Vanti inició y hubo respuesta afirmativa
            tipo_respuesta = random.choices(['Consulta', 'Reclamo', 'Solicitud de Información', 'Felicitación'], weights=[40, 30, 20, 10], k=1)[0]
            contacto_respuesta = {
                'ID Contacto': len(contactabilidad) + 1,
                'ID Canal': id_canal,
                'Fecha': random_date(contacto['Fecha'], DATE_END),
                'Canal': canal_tipo,
                'Tipo de contacto': tipo_respuesta,
                'Iniciador del Contacto': 'Cliente',
                'Respuesta': random.choices(list(DISTRIBUCIONES['respuesta'].keys()), weights=list(DISTRIBUCIONES['respuesta'].values()), k=1)[0],
                'Satisfacción': random.choices(list(DISTRIBUCIONES['satisfaccion'].keys()), weights=list(DISTRIBUCIONES['satisfaccion'].values()), k=1)[0]
            }
            contactabilidad.append(contacto_respuesta)
    return pd.DataFrame(contactabilidad)

df_contactabilidad = generar_contactabilidad(NUM_CONTACTS, df_clientes, df_canales)

# Generación de respuestas
def generar_respuestas(df_contactabilidad):
    respuestas = []
    for _, contacto in df_contactabilidad.iterrows():
        probabilidad_respuesta = {'Email': 80, 'Teléfono': 85, 'Dirección': 20, 'WhatsApp': 90}[contacto['Canal']]
        if random.random() < (probabilidad_respuesta / 100):
            respuesta = {
                'ID Respuesta': len(respuestas) + 1,
                'ID Contacto': contacto['ID Contacto'],
                'Respuesta': fake.sentence(nb_words=10),
                'Fecha de respuesta': random_date(contacto['Fecha'], DATE_END)
            }
            respuestas.append(respuesta)
    return pd.DataFrame(respuestas)

df_respuestas = generar_respuestas(df_contactabilidad)

# Generación de tabla adicional para asociar sociedades de Vanti con los departamentos
def generar_sociedad_departamento():
    sociedades = [
        ('Vanti S.A. ESP', 'Bogotá'),
        ('Gas Natural del Cesar S.A. ESP', 'Cesar'),
        ('Gas Natural Cundiboyacense S.A. ESP', 'Cundinamarca'),
        ('Gas Natural del Oriente S.A. ESP', 'Santander')
    ]
    return pd.DataFrame(sociedades, columns=['Sociedad', 'Departamento'])

df_sociedad_departamento = generar_sociedad_departamento()


## Duplicidades de registros para Algoritmo Jaro Winkler

In [ ]:
import pandas as pd
import random
import string
from faker import Faker
from datetime import datetime, timedelta

# Configuración de Faker
fake = Faker('es_CO')
Faker.seed(1)

# Parámetros para generar duplicados sintéticos
PERCENT_DUPLICATES = 0.1  # Porcentaje de clientes que serán duplicados

# Variable para llevar el último ID de cliente
ultimo_id = df_clientes['ID Cliente'].max()  # Asume que los IDs son numéricos y consecutivos

# Función para introducir variaciones en los datos
def introducir_errores_documento(identificacion):
    identificacion = str(identificacion)
    variacion = random.choice([1, 2, 3])
    if variacion == 1:
        return identificacion + random.choice(string.ascii_letters)
    elif variacion == 2:
        return identificacion[:4] + ' ' + identificacion[4:]
    elif variacion == 3:
        return identificacion + str(random.randint(0, 9))
    return identificacion


def introducir_errores_telefono(telefono):
    telefono = str(telefono)
    variacion = random.choice([1, 2, 3])
    if variacion == 1:
        return telefono.replace(' ', '')
    elif variacion == 2:
        return telefono[:6] + str(random.randint(0, 9)) + telefono[7:]
    elif variacion == 3:
        return telefono[:4] + telefono[5:]
    return telefono


def introducir_errores_email(email):
    email = str(email)
    variacion = random.choice([1, 2, 3])
    if variacion == 1:
        return email.replace('.', '', 1)
    elif variacion == 2:
        return email + random.choice(string.ascii_letters)
    elif variacion == 3:
        return email.replace('@', random.choice(string.ascii_letters) + '@')
    return email


def introducir_errores_direccion(direccion):
    direccion = str(direccion)
    variacion = random.choice([1, 2, 3])
    if variacion == 1:
        return direccion.replace(' #', ' No.')
    elif variacion == 2:
        return direccion + ', Apt ' + str(random.randint(1, 10))
    elif variacion == 3:
        return direccion.split(',')[0]
    return direccion


def generar_duplicados_sinteticos(df_clientes, df_canales, porcentaje_duplicados):
    global ultimo_id  # Para usar y actualizar el último ID de cliente
    num_duplicados = int(len(df_clientes) * porcentaje_duplicados)
    duplicados = []

    for _ in range(num_duplicados):
        cliente_original = df_clientes.sample(1).iloc[0]
        cliente_duplicado = cliente_original.copy()

        # Asignar un nuevo ID único y consecutivo al cliente duplicado
        ultimo_id += 1
        cliente_duplicado['ID Cliente'] = ultimo_id

        # Variar algunos campos del cliente duplicado
        cliente_duplicado['Identificación'] = introducir_errores_documento(cliente_original['Identificación'])
        cliente_duplicado['Nombre'] = fake.name() if random.random() < 0.3 else cliente_original['Nombre']
        cliente_duplicado['Sexo'] = cliente_original['Sexo']
        cliente_duplicado['Nivel educativo'] = cliente_original['Nivel educativo']
        cliente_duplicado['Ocupación'] = cliente_original['Ocupación']
        cliente_duplicado['Estado civil'] = cliente_original['Estado civil']
        cliente_duplicado['Ciudad'] = cliente_original['Ciudad']
        cliente_duplicado['Departamento'] = cliente_original['Departamento']
        duplicados.append(cliente_duplicado)

        # Variar también los canales de contacto asociados
        canales_cliente = df_canales[df_canales['ID Cliente'] == cliente_original['ID Cliente']]
        for _, canal in canales_cliente.iterrows():
            canal_duplicado = canal.copy()
            canal_duplicado['ID Cliente'] = cliente_duplicado['ID Cliente']  # Asignar el nuevo ID del cliente
            if canal['Tipo'] == 'Teléfono' or canal['Tipo'] == 'WhatsApp':
                canal_duplicado['Valor'] = introducir_errores_telefono(canal['Valor'])
            elif canal['Tipo'] == 'Email':
                canal_duplicado['Valor'] = introducir_errores_email(canal['Valor'])
            elif canal['Tipo'] == 'Dirección':
                canal_duplicado['Valor'] = introducir_errores_direccion(canal['Valor'])
            canal_duplicado['Fecha de actualización'] = pd.to_datetime(canal['Fecha de actualización']) + timedelta(days=random.randint(1, 30))
            canal_duplicado['Validación'] = random.choices([True, False], weights=[85, 15], k=1)[0]
            canal_duplicado['Consentimiento'] = random.choices([True, False], weights=[80, 20], k=1)[0]
            df_canales = pd.concat([df_canales, pd.DataFrame([canal_duplicado])], ignore_index=True)

    # Añadir los duplicados al DataFrame original
    df_clientes = pd.concat([df_clientes, pd.DataFrame(duplicados)], ignore_index=True)
    return df_clientes, df_canales

# Generar duplicados sintéticos
df_clientes, df_canales = generar_duplicados_sinteticos(df_clientes, df_canales, PERCENT_DUPLICATES)



## Carga a BigQuery

In [ ]:
from google.cloud import bigquery

# Enviar las tablas a BigQuery
# Especificar el ID del proyecto y el conjunto de datos
project_id = 'vanti-poc-440213'
dataset_id = 'data_poc'

# Inicializar el cliente de BigQuery
client = bigquery.Client(project=project_id)

# Definir las tablas y los DataFrames correspondientes
tables = {
    'raw_clientes': df_clientes,
    'raw_canales_de_contacto': df_canales,
    'raw_contactabilidad': df_contactabilidad,
    'raw_respuestas': df_respuestas,
    'raw_sociedades': df_sociedad_departamento
}

# Configurar el trabajo de carga
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE  # Sobrescribir las tablas si ya existen

# Escribir cada DataFrame en una tabla de BigQuery
for table_name, df in tables.items():
    table_id = f"{project_id}.{dataset_id}.{table_name}"
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()  # Esperar a que el trabajo se complete
    print(f"Los datos se han cargado en la tabla {table_id}")